Determine a function f such that the value of the pension savings in 30 years can be expressed as $V_{30} = f(μ,σ,p,c,Z_1,...,Z_{30})$. Then simulate a sample of suitable size n from the distribution of $(Z_1, . . . , Z_{30})$ and use this sample to determine the empirical distribution $F_n$ of $V_{30}$ for a range of values of the parameters μ, σ, p, c.

In [2]:
%run HW2.ipynb


NameError: name 'R1' is not defined

NameError: name 'R1' is not defined